# Country Club SQL Project Part 2

In [15]:
#Import associated packges in order to connect to SQLite instance
from sqlalchemy import create_engine
import pandas as pd

In [16]:
#Create engine to connect to the database file
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [50]:
pd.read_sql_query('SELECT * FROM Facilities', engine).head()

facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000

In [92]:
pd.read_sql_query('SELECT * FROM Bookings', engine).head()

bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1

In [20]:
pd.read_sql_query('SELECT * FROM Members', engine).head()

memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05

In [132]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members!

pd.read_sql_query('SELECT f.name AS facility, (COUNT(b.memid <> 0) * f.membercost + COUNT(b.memid = 0)*f.guestcost) AS total_revenue FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid GROUP BY f.name HAVING total_revenue < 1000;', engine)

Empty DataFrame
Columns: [facility, total_revenue]
Index: []

In [118]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

pd.read_sql_query(' SELECT m1.surname AS member_lastname , m1.firstname As member_firstname, m2.surname AS member_recommended_lastname, m2.firstname AS member_recommended_firstname FROM Members AS m1 INNER JOIN Members AS m2 ON m1.memid = m2.recommendedby ORDER BY member_recommended_lastname, member_recommended_firstname;', engine)

member_lastname member_firstname member_recommended_lastname  \
0         Stibbons           Ponder                       Bader   
1         Stibbons           Ponder                       Baker   
2          Farrell           Jemima                       Baker   
3           Rownam              Tim                      Boothe   
4            Smith           Darren                     Butters   
5            Baker          Timothy                      Coplin   
6            Smith            Tracy                     Crumpet   
7         Joplette           Janice                        Dare   
8          Butters           Gerald                     Genting   
9          Purview        Millicent                        Hunt   
10        Joplette           Janice                       Jones   
11           Jones            David                       Jones   
12           Smith           Darren                    Joplette   
13           Smith           Darren                   Mackenzie   
14           Smith           Darren                        Owen   
15         Farrell           Jemima                      Pinker   
16           Smith            Tracy                     Purview   
17         Genting          Matthew                      Rumney   
18           Bader         Florence                      Sarwin   
19           Smith           Darren                       Smith   
20           Tracy           Burton                    Stibbons   
21           Smith            Tracy           Worthington-Smyth   

   member_recommended_firstname  
0                      Florence  
1                          Anne  
2                       Timothy  
3                           Tim  
4                        Gerald  
5                          Joan  
6                         Erica  
7                         Nancy  
8                       Matthew  
9                          John  
10                        David  
11                      Douglas  
12                       Janice  
13                         Anna  
14                      Charles  
15                        David  
16                    Millicent  
17                    Henrietta  
18                    Ramnaresh  
19                         Jack  
20                       Ponder  
21                        Henry

In [119]:
#Q12: Find the facilities with their usage by member, but not guests

pd.read_sql_query('SELECT m.firstname || " " || m.surname AS member_name, f.name As facility, COUNT(f.facid) AS facility_usage FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid INNER JOIN Members AS m ON b.memid = m.memid WHERE m.memid <> 0 GROUP BY m.surname , f.name ORDER BY member_name;', engine)

member_name         facility  facility_usage
0    Anna Mackenzie  Badminton Court              30
1    Anna Mackenzie   Massage Room 1               1
2    Anna Mackenzie       Pool Table              70
3    Anna Mackenzie    Snooker Table               7
4    Anna Mackenzie     Squash Court               2
..              ...              ...             ...
166      Tim Rownam   Tennis Court 1               6
167      Tim Rownam   Tennis Court 2               6
168   Timothy Baker  Badminton Court              17
169   Timothy Baker   Massage Room 1              27
170     Tracy Smith   Tennis Court 1              52

[171 rows x 3 columns]

In [114]:
#Q13: Find the facilities usage by month, but not guests
#Code for this statement found on https://stackoverflow.com/questions/57023135/how-to-convert-the-date-time-to-date-in-sqlite
pd.read_sql_query('SELECT strftime("%m", b.starttime) AS month, f.name As facility, COUNT(f.facid) AS facility_usage FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid INNER JOIN Members AS m ON b.memid = m.memid WHERE m.memid <> 0 GROUP BY strftime("%m", b.starttime), f.name ORDER BY facility_usage DESC;', engine)  

month         facility  facility_usage
0     09       Pool Table             408
1     08       Pool Table             272
2     09    Snooker Table             199
3     09     Table Tennis             194
4     09   Massage Room 1             191
5     09  Badminton Court             161
6     08    Snooker Table             154
7     08   Massage Room 1             153
8     08     Table Tennis             143
9     08  Badminton Court             132
10    09   Tennis Court 1             132
11    09   Tennis Court 2             126
12    08   Tennis Court 1             111
13    08   Tennis Court 2             109
14    07       Pool Table             103
15    09     Squash Court              87
16    08     Squash Court              85
17    07   Massage Room 1              77
18    07    Snooker Table              68
19    07   Tennis Court 1              65
20    07  Badminton Court              51
21    07     Table Tennis              48
22    07   Tennis Court 2              41
23    07     Squash Court              23
24    09   Massage Room 2              14
25    08   Massage Room 2               9
26    07   Massage Room 2               4